In [16]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import pandas as pd

orders = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/innomatics/orders.csv')
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [18]:
users = pd.read_json('/content/drive/MyDrive/Colab Notebooks/innomatics/users.json')
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [19]:
import sqlite3

# Create in-memory SQLite DB
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Read SQL file
with open('/content/drive/MyDrive/Colab Notebooks/innomatics/restaurants.sql', 'r') as f:
    sql_script = f.read()

cursor.executescript(sql_script)

restaurants = pd.read_sql('SELECT * FROM restaurants', conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [20]:
df = orders.merge(users, on='user_id', how='left')

In [21]:
df = df.merge(restaurants, on='restaurant_id', how='left')

In [22]:
df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [23]:
df.to_csv(
    '/content/drive/MyDrive/Colab Notebooks/innomatics/final_food_delivery_dataset.csv',
    index=False
)

In [24]:
import pandas as pd

df = pd.read_csv(
    '/content/drive/MyDrive/Colab Notebooks/innomatics/final_food_delivery_dataset.csv'
)

df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [26]:
df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [28]:
df[df['membership'] == 'Gold'] \
.groupby('city')['total_amount'] \
.sum() \
.sort_values(ascending=False)


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [29]:
df.groupby('cuisine')['total_amount'] \
.mean() \
.sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [30]:
user_spend = df.groupby('user_id')['total_amount'].sum()
user_spend[user_spend > 1000].nunique()


2543

In [31]:
bins = [0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0']

df['rating_range'] = pd.cut(df['rating'], bins=bins, labels=labels)

df.groupby('rating_range')['total_amount'] \
.sum() \
.sort_values(ascending=False)


/tmp/ipython-input-1965542159.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('rating_range')['total_amount'] \


,total_amount
rating_range,
4.6–5.0,2197030.75
3.0–3.5,2136772.70
4.1–4.5,1960326.26
3.6–4.0,1717494.41


In [32]:
df[df['membership'] == 'Gold'] \
.groupby('city')['total_amount'] \
.mean() \
.sort_values(ascending=False)


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [33]:
restaurant_count = df.groupby('cuisine')['restaurant_id'].nunique()
revenue = df.groupby('cuisine')['total_amount'].sum()

summary = pd.concat([restaurant_count, revenue], axis=1)
summary.columns = ['restaurant_count', 'total_revenue']

summary.sort_values(
    by=['restaurant_count', 'total_revenue'],
    ascending=[True, False]
)


,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [34]:
gold_orders = df[df['membership'] == 'Gold'].shape[0]
total_orders = df.shape[0]

round((gold_orders / total_orders) * 100)


50

In [35]:
stats = df.groupby('restaurant_name_y').agg(
    avg_order_value=('total_amount', 'mean'),
    total_orders=('order_id', 'count')
)

stats[stats['total_orders'] < 20] \
.sort_values(by='avg_order_value', ascending=False) \
.head(5)


,avg_order_value,total_orders
restaurant_name_y,,
Restaurant_294,1040.222308,13
Restaurant_262,1029.473333,18
Restaurant_77,1029.180833,12
Restaurant_193,1026.306667,15
Restaurant_7,1002.140625,16


In [36]:
df.groupby(['membership', 'cuisine'])['total_amount'] \
.sum() \
.sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [37]:
df['order_date'] = pd.to_datetime(df['order_date'])
df['quarter'] = df['order_date'].dt.to_period('Q')

df.groupby('quarter')['total_amount'] \
.sum() \
.sort_values(ascending=False)


/tmp/ipython-input-4276944275.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['order_date'] = pd.to_datetime(df['order_date'])


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [38]:
df[df['membership'] == 'Gold'].shape[0]


4987

In [39]:
round(
    df[df['city'] == 'Hyderabad']['total_amount'].sum()
)


1889367

In [40]:
df['user_id'].nunique()


2883

In [41]:
round(
    df[df['membership'] == 'Gold']['total_amount'].mean(),
    2
)


np.float64(797.15)

In [42]:
df[df['rating'] >= 4.5].shape[0]


3374

In [43]:
top_city = (
    df[df['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .idxmax()
)

top_city


'Chennai'

In [44]:
df[
    (df['membership'] == 'Gold') &
    (df['city'] == top_city)
].shape[0]


1337